John Ryan Kivela, MA

UMass Boston

College of Management

MSIS - 672 Final Project

Data Preprocessing

4/20/24

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.impute import SimpleImputer

In [2]:
## Load data
df = pd.read_csv('output_file.csv')

In [3]:
df.head()

,ID,Customer_ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,23,Scientist,19114.12,1824.843333,3,4,3,...,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.642682,Low_spent_Small_value_payments,186.2667021
1,0x160b,CUS_0xd40,October,24,Scientist,19114.12,1824.843333,3,4,3,...,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380,High_spent_Medium_value_payments,361.4440039
2,0x160c,CUS_0xd40,November,24,Scientist,19114.12,1824.843333,3,4,3,...,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.233938,Low_spent_Medium_value_payments,264.6754462
3,0x160d,CUS_0xd40,December,24,Scientist,19114.12,NaN,3,4,3,...,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.082511,High_spent_Medium_value_payments,343.8268732
4,0x1616,CUS_0x21b1,September,28,NaN,34847.84,3037.986667,2,4,6,...,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018,High_spent_Large_value_payments,485.2984337


In [4]:
df.shape

(40195, 25)

In [5]:
df.columns

Index(['ID', 'Customer_ID', 'Month', 'Age', 'Occupation', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance'],
      dtype='object')

In [6]:
df.dtypes

ID                           object
Customer_ID                  object
Month                        object
Age                           int64
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                   int64
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly     float64
Payment_Behaviour            object
Monthly_Balance              object
dtype: object

In [7]:
# Wrangle Data

In [8]:
## Columns to drop later
columns_to_drop = ['ID',
                   'Customer_ID',
                   'Occupation',
                   'Credit_History_Age',
                   'Type_of_Loan',
                   'Month',
                   'Age',
                   'Payment_Behaviour'] # This was hard to let go, but I think it could be useful in a future assessment.

In [9]:
## Convert 'Monthly_Balance' column to float, handle errors by coercing to NaN
df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')

## Convert Num of delayed payments to Int64
df["Num_of_Delayed_Payment"] = df["Num_of_Delayed_Payment"].astype('Int64')

# Convert Payment of min amount to boolean
df["Payment_of_Min_Amount"] = df["Payment_of_Min_Amount"].map({'Yes': True, 'No': False})

In [10]:
# Drop those Columns from above like they're hot
df = df.drop(columns = columns_to_drop)

In [11]:
# Deal with NA values
## Identify Variable with NaN values
def find_columns_with_nan(df):
    columns_with_nan = [col for col in df.columns if df[col].isna().any()]
    return columns_with_nan

### Identify Variable with NaN values
columns_with_nan = find_columns_with_nan(df)
print("Columns with NaN values:", columns_with_nan)

Columns with NaN values: ['Monthly_Inhand_Salary', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Payment_of_Min_Amount', 'Amount_invested_monthly', 'Monthly_Balance']


In [12]:
# Impute values for NA with numbers
## Columns to impute
columns_to_impute = ['Monthly_Inhand_Salary',
                     'Num_of_Delayed_Payment',
                     'Changed_Credit_Limit',
                     'Num_Credit_Inquiries',
                     'Amount_invested_monthly',
                     'Monthly_Balance'
                     ]

## make a function
def impute_selected_columns(df, columns_to_impute):
    # Use SimpleImputer with strategy='mean'
    imputer = SimpleImputer(strategy='mean')
    
    # Select columns to impute
    df_to_impute = df[columns_to_impute]
    
    # Impute NaN values in selected columns
    df_imputed = pd.DataFrame(imputer.fit_transform(df_to_impute), 
                              columns = columns_to_impute)
    
    # Update original DataFrame with imputed values
    df[columns_to_impute] = df_imputed
    
    # return the data frame
    return df 

## Impute missing values for selected columns
df = impute_selected_columns(df, 
                             columns_to_impute)

In [13]:
## Categorical variable wrangling

### Replace NaN values in 'Payment_of_Min_Amount' with 'unknown'
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].fillna("Unknown")

In [14]:
# Check for NA
columns_with_nan = find_columns_with_nan(df)
print("Columns with NaN values:", columns_with_nan)

Columns with NaN values: []


In [15]:
# Covert to Int64
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].astype('int64')

# Covert to Int64
df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].astype('int64')

In [16]:
## One-hot encoding
### List of variables to encode
variables_to_encode = ['Payment_of_Min_Amount']

### Perform one-hot encoding for the variables in variables_to_encode
df_hot = pd.get_dummies(df, columns=variables_to_encode)

In [17]:
df_hot.dtypes

Annual_Income                    float64
Monthly_Inhand_Salary            float64
Num_Bank_Accounts                  int64
Num_Credit_Card                    int64
Interest_Rate                      int64
Num_of_Loan                        int64
Delay_from_due_date                int64
Num_of_Delayed_Payment             int64
Changed_Credit_Limit             float64
Num_Credit_Inquiries               int64
Credit_Mix                        object
Outstanding_Debt                 float64
Credit_Utilization_Ratio         float64
Total_EMI_per_month              float64
Amount_invested_monthly          float64
Monthly_Balance                  float64
Payment_of_Min_Amount_False         bool
Payment_of_Min_Amount_True          bool
Payment_of_Min_Amount_Unknown       bool
dtype: object

In [18]:
# Write DataFrame to CSV file
df_hot.to_csv('Kivela - Final Exam PreProcessed.csv', 
              index=False)